This example aims to have two target orbits, and one observer orbit. The observer is tasked with monitoring the two targets to minimize the trace covariance of the state over the simulation horizon. We use a PPO agent that is rewarded for minimizing the trace covariance of the target states to the best of its ability.

In [1]:
%matplotlib widget
import sys
sys.path.append("../src")

import os
import dill
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt

from SensorTasking import SpaceEnv, run_myopic_policy, func
from SensorTasking.observation_model import DummyModel, ApparentMag
from SensorTasking.observation_spaces import Type1, Type2

from data_util.target_generation import TargetGenerator

from stable_baselines3.common.env_checker import check_env

In [2]:
# mass parameter of earth-moon system
mu = 1.215058560962404e-02
# apparent magnitude of sun
ms = -26.4
# specular reflection coefficient
aspec = 0
# diffuse reflection coefficient
adiff = 0.2
# diameter of target (LU)
d = 0.001 / 384400
# earth radius (LU)
rearth = 6371 / 384400
# moon radius (LU)
rmoon = 1737.4 / 384400

params = {
    "mu":mu,
    "ms":ms,
    "aspec":aspec,
    "adiff":adiff,
    "d":d,
    "rearth":rearth,
    "rmoon":rmoon
}

# L2 Halo orbit, period = 3.225 TU  (Target orbit)
x0_1 = np.array([
        1.1540242813087864,
        0.0,
        -0.1384196144071876,
        4.06530060663289e-15,
        -0.21493019200956867,
        8.48098638414804e-15
    ])

# L1 lyapunov, period = 3.225 TU (target orbit)
x0_2 = np.array([
        0.8027692908754149,
        0.0,
        0.0,
        -1.1309830924549648e-14,
        0.33765564334938736,
        0.0
    ])


# 1:1 L2 Lyapunov, period = 6.45 TU (observer orbit)
x0_3 = np.array([
        0.9982702689023665,
        0.0,
        0.0,
        -2.5322340091977996e-14,
        1.5325475708886613,
        0.0
    ])



In [3]:
splines = []
splines.append(dill.load(open(f"../data/splines/l2halo_spl.pickle", "rb")))
splines.append(dill.load(open(f"../data/splines/l1lyap_spl.pickle", "rb")))
tg = TargetGenerator([x0_1, x0_2], periods=[3.225, 3.225], splines = splines)

splines = [dill.load(open(f"../data/splines/l2lyap_spl.pickle", "rb"))]
ag = TargetGenerator([x0_3], periods = [6.45], splines=splines)

In [4]:
log_dir = "/Users/malavpatel/Research/SensorTasking/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

In [5]:
targets = tg.gen_phased_ics([1,1], gen_P=True)
agents = ag.gen_phased_ics([1], gen_P=False)
maxsteps = 215
tstep = 0.030
# obs_model = ApparentMag(params, tstep, sun_phasing=np.pi/2)
obs_model = DummyModel()
obs_class = Type1()
policytype = "MultiInputPolicy"

env = SpaceEnv(agents, targets, obs_model, maxsteps, tstep, obs_class=obs_class)

check_env(env)


In [14]:
x = np.array([np.pi])
y = numpy.matlib.repmat([ [2], [1], [1], [2]], 54, 1)

In [15]:
func(x, y, env)

terminated episode


-0.010779093989143534